In [204]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, InputLayer, BatchNormalization

In [86]:
#settings:
input_dim = 388
strDataDir = r'/project/hackathon/hackers12/shared'
dfNameFile = pd.read_excel(os.path.join(strDataDir,'Clean_data.xlsx'), sheet_name = 'Names File')
dfNameFile.sort_values(by=['group_number'], inplace=True)
dfNameFile = dfNameFile[dfNameFile['group_number'] <= 4]
dfData = pd.read_excel(os.path.join(strDataDir,'Clean_data.xlsx'), sheet_name = 'Isotope corrected')
dfDataNamed = dfData[dfData.Group != 'Unknown']
dfDataNamed.sort_values(by=['Group'], inplace=True)
dfDataNorm = pd.read_csv(os.path.join(strDataDir,'Normed_Data.csv'))
dfDataNormNamed = dfDataNorm[dfDataNorm['Group'] != 'Unknown']

#dfData.sort_values(by=['Named Data'], inplace=True)

lSamples = dfNameFile['sample_name'].tolist()
lNameCols = ['Name','Group','Precursor','Fragment']

dfTestData = pd.read_csv(os.path.join(strDataDir,'samples_in_testing_set.csv'))
lTestSamples = dfTestData.sample_name.tolist()

lTrainSamples = [strSample for strSample in lSamples if not strSample in lTestSamples]
#dfValidationData = pd.read_csv(os.path.join(strDataDir,'samples_in_validation_set.csv'))
#lValidationSamples = 


/project/bioinformatics/DLLab/shared/CondaEnvironments/Alex365tfGPU/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [66]:
print(lTrainSamples.__len__(), lTestSamples.__len__())
print(lSamples.__len__())

78 9
87


In [67]:
dfDataNormNamed.shape

(388, 95)

In [164]:
y = []
X = []
for strSample in lTrainSamples:
    dfSampleName = dfNameFile[dfNameFile.sample_name == strSample]
    if dfSampleName.shape[0] != 1:
        raise ValueError('Duplicate Name')
    sSample = dfSampleName.iloc[0]
    if sSample['group_number'] == 1 or sSample['group_number'] == 2:
        listOneHot = [1,0,0]
    if sSample['group_number'] == 3:
        listOneHot = [0,1,0]
    if sSample['group_number'] == 4:
        listOneHot = [0,0,1]
    y.append(listOneHot)
    X.append(dfDataNormNamed[strSample].tolist())
y = np.array(y)
X = np.array(X)

In [165]:
print(X.shape, y.shape)

(78, 388) (78, 3)


In [176]:
def fMakeModel():
    optimizer = keras.optimizers.Adam()
    model = keras.models.Sequential()
    model.add(Dense(64, activation='relu', input_dim=388))
    model.add(Dropout(0.5))
    #model.add(Dense(4, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    #dctHistory = model.fit(X, y, epochs=50, verbose=0)

In [187]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X, y.argmax(axis=1))
lValAcc = []
for train_index, validation_index in skf.split(X, y.argmax(axis=1)):
    Xtrain = X[train_index]
    ytrain = y[train_index]
    Xvalidation = X[validation_index]
    yvalidation = y[validation_index]
    model = fMakeModel()
    class_weight = {0:.5,1:1,2:1}
    dctHistory = model.fit(Xtrain, ytrain, 
                           epochs=50,
                           verbose=0,
                           validation_data=[Xvalidation, yvalidation],
                           class_weight = class_weight)
    lValAcc.append(np.max(dctHistory.history['val_acc']))
print(lValAcc)

[0.5555555820465088, 0.5555555820465088, 0.5, 0.5, 0.875, 0.375, 0.75, 0.875, 0.6666666865348816, 0.8333333134651184]


In [188]:
np.mean(lValAcc)

0.6486111164093018

In [218]:
strModelOutput = os.path.join(strDataDir, 'alex_models')
os.path.exists(strModelOutput)

True

In [ ]:
layers = [0,1,2,3,4]
units = [8,16,32,64,128,256]
drop_out = [0, .2, .5]
batchnormalization = [True, False]
intC = 0
while True:
    try:
        intLayers = np.random.choice(layers)
        optimizer = keras.optimizers.Adam()
        model = keras.models.Sequential()
        bBatchnormalization = np.random.choice(batchnormalization)
        #model.add(InputLayer(input_shape = (388,)))
        for i in range(intLayers):
            if i == 0:
                model.add(Dense(np.random.choice(units), activation='relu', input_dim=388))
            else:
                model.add(Dense(np.random.choice(units), activation='relu'))
            if bBatchnormalization:
                model.add(BatchNormalization())
            dDropout = np.random.choice(drop_out)
            if dDropout != 0:
                model.add(Dropout(rate=dDropout))
        model.add(Dense(3, activation='softmax'))
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        tenInitialWeights = model.get_weights()
        model.save(os.path.join(strModelOutput, 'model%i.h5')%intC)
        dfPerformance = pd.DataFrame()
        intFold = 0
        for train_index, validation_index in skf.split(X, y.argmax(axis=1)):
            Xtrain = X[train_index]
            ytrain = y[train_index]
            Xvalidation = X[validation_index]
            yvalidation = y[validation_index]
            #model = fMakeModel()
            class_weight = {0:.5,1:1,2:1}
            dctHistory = model.fit(Xtrain, ytrain, 
                                   epochs=100,
                                   verbose=0,
                                   validation_data=[Xvalidation, yvalidation],
                                   class_weight = class_weight)
            for key in dctHistory.history:
                dfPerformance['%s%i'%(key, intFold)] = dctHistory.history[key]
            #print(np.max(dctHistory.history['val_acc']))
            model.set_weights(tenInitialWeights)
            intFold+=1
        dfPerformance.to_csv((os.path.join(strModelOutput, 'model%i.csv')%intC))
        intC+=1
        print(intC)
    except:
        pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
